# 태스크 6: LangGraph와 Bedrock 모델 통합

이 노트북에서는 에이전트가 사용할 수 있는 도구를 사용하여 작업 순서를 결정하고, 이를 구현하는 계획 및 실행 에이전트를 사용하는 방법을 알아보겠습니다. 

특정 애플리케이션은 사용자의 질문에 답변하기 위해 언어 모델과 다양한 유틸리티에 대한 적응 가능한 호출 순서를 요구합니다. LangChain 에이전트 인터페이스는 유연하며, 외부 도구와 LLM의 추론을 통합할 수 있습니다. 에이전트는 사용자 입력에 따라 사용할 도구를 선택할 수 있습니다. 에이전트는 여러 도구를 사용할 수 있으며 한 도구의 출력을 다음 도구의 입력으로 활용할 수 있습니다.

## 태스크 6.1: 환경 설정

이 태스크에서는 환경을 설정합니다.

In [9]:
!pip show langchain langchain-core langgraph

Name: langchain
Version: 0.3.27
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /opt/conda/lib/python3.12/site-packages
Requires: langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: jupyter_ai_magics, langchain-community
---
Name: langchain-core
Version: 0.3.79
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /opt/conda/lib/python3.12/site-packages
Requires: jsonpatch, langsmith, packaging, pydantic, PyYAML, tenacity, typing-extensions
Required-by: amazon_sagemaker_jupyter_ai_q_developer, langchain, langchain-aws, langchain-community, langchain-text-splitters, langgraph, langgraph-checkpoint, langgraph-prebuilt
---
Name: langgraph
Version: 0.6.10
Summary: Building stateful, multi-actor applications with LLMs
Home-page: 
Author: 
Author-email: 
License-Expression: MIT
Location: /opt/conda/l

In [31]:
from typing import Annotated
import pandas as pd
import numexpr

# LangChain / LangGraph imports
from langchain.tools import tool
from langchain_aws import ChatBedrock
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langgraph.prebuilt import ToolNode, create_react_agent
from langgraph.graph import StateGraph, END

In [40]:
# ----------------------------------------
# 🧰 Tools 정의
# ----------------------------------------

@tool
def lookup_price(item_name: str) -> str:
    """Lookup the price of a product by its name from the CSV file."""
    df = pd.read_csv("sales.csv")
    result = df[df["item"] == item_name]
    if result.empty:
        return f"'{item_name}' not found."
    return str(result["price"].values[0])


@tool
def calculator(expression: str) -> str:
    """Evaluate a mathematical expression and return the result as a string."""
    try:
        result = numexpr.evaluate(expression)
        return str(result)
    except Exception as e:
        return f"Error evaluating expression: {e}"


@tool
def echo_tool(text: str) -> str:
    """Echo back the provided text."""
    return f"Echo: {text}"

In [41]:
# ----------------------------------------
# 🧩 Agent Node 생성
# ----------------------------------------

# Bedrock 모델 (또는 호환 모델)
llm = ChatBedrock(
    model_id="amazon.nova-lite-v1:0",
    region="us-east-1"
)

In [42]:
# 사용할 도구 리스트
tools = [lookup_price, calculator, echo_tool]

# create_react_agent: LLM이 ToolNode를 통해 도구를 호출하도록 자동 구성
agent = create_react_agent(llm, tools)

In [43]:
print("=== LangGraph ReAct Agent Example ===\n")
user_input = "사용자 질문:How much will it cost to buy 3 units of P002 and 5 units of P003?"

# ✅ agent 자체가 실행 가능한 Graph입니다
result = agent.invoke({"messages": [HumanMessage(content=user_input)]})

print("\n🤖 Agent Response:")
print(result["messages"][-1].content)

=== LangGraph ReAct Agent Example ===


🤖 Agent Response:
<thinking> The 'echo_tool' confirms that the item names are being correctly passed to the tools. However, the 'lookup_price' tool still seems to be malfunctioning. I will need to inform the user about the issue and suggest they check back later or contact support for assistance. </thinking>

<response> I'm sorry, but I'm currently unable to look up the prices of the items due to a technical issue with the 'lookup_price' tool. Please check back later or contact support for assistance. </response>


## 태스크 6.2: Synergizing Reasoning and Acting in Language Models 프레임워크

이 태스크에서 ReAct 프레임워크는 대규모 언어 모델이 외부 도구와 상호 작용하여 보다 더 정확하고 사실에 기반한 응답을 제공하는 추가 정보를 얻을 수 있게 합니다.

대규모 언어 모델은 추론에 대한 설명과 작업별 응답을 교대로 생성할 수 있습니다.

추론 설명을 생성하면 모델이 실행 계획을 추론, 모니터링 및 수정할 수 있고, 예상치 못한 시나리오도 처리할 수도 있습니다. 실행 단계에서 모델은 지식 기반 또는 환경과 같은 외부 소스와 인터페이스를 통해 정보를 획득할 수 있습니다.

다음 셀에서는 Langchain 프레임워크 내에서 도구 역할을 하는 `calculator` 함수를 정의합니다. 이 도구는 언어 모델이 Python의 numexpr 라이브러리를 사용해 주어진 식을 평가하여 수학적 계산을 수행할 수 있게 합니다. 이 도구는 식이 유효하지 않은 경우를 처리하도록 설계되었습니다. 이 경우, 이 도구는 계산에 대한 접근 방식 재고하도록 모델에 요청합니다.

In [15]:
@tool
def lookup_price(query: str) -> str:
    """Look up a product price from a CSV file.
    Expected query: product_id (or line with product_id on first line)
    The original notebook used a CSV file; adapt the path as needed.
    """
    csv_path = os.environ.get("PRICES_CSV", "prices.csv")
    prices = {}
    if not os.path.exists(csv_path):
        return f"Price CSV not found at {csv_path}"
    try:
        with open(csv_path, newline='', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                prices[row.get('product_id') or row.get('id')] = row.get('price')
    except Exception as e:
        return f"Failed to read CSV: {e}"

    qstr = query.split('\n')[0].strip()
    price = prices.get(qstr)
    if price is None:
        return f"Price for product {qstr} is not available"
    return f"Price of product {qstr} is {price}"

다음 셀에서는 Langchain 프레임워크 내에서 도구 역할을 하는 `calculator` 함수를 정의합니다. 이 도구를 사용하면 언어 모델에서 Python의 numexpr 라이브러리를 통해 주어진 표현식을 평가하여 수학적 계산을 수행할 수 있습니다. 표현식이 유효하지 않은 사례가 처리되도록 도구가 설계되었습니다. 해당 사례에서는 계산에 대한 접근 방식 재고를 모델에 요청합니다.

In [16]:
@tool
def calculator(expression: str) -> str:
    """Safely evaluate a math expression and return the result as string.
    Uses numexpr for basic arithmetic.
    """
    try:
        # strip natural-language wrappers if user sent 'compute: 5*4' etc.
        expr = expression.strip()
        # Evaluate using numexpr to limit operations
        result = numexpr.evaluate(expr, global_dict={}, local_dict={})
        return str(result)
    except Exception:
        return "Error evaluating expression. Please provide a valid math expression."


In [22]:
@tool
def echo_tool(text: str) -> str:
    """Lookup the price of a product by its name from the CSV file."""
    return text

# Bind tools to the LLM
tools = [lookup_price, calculator, echo_tool]
tools_by_name = {t.name: t for t in tools}
llm_with_tools = chat_model.bind_tools(tools)

In [26]:
# --- Step 3: Define shared state type ---
class MessagesState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]
    llm_calls: int

# --- Step 4: Define nodes ---
# Model node: ask the model to decide next step / produce a message
SYSTEM_PROMPT = "You are a helpful assistant. Use the available tools when needed."

def llm_node(state: MessagesState):
    """Call the LLM (with tools) and append its response to state['messages']"""
    # Build messages: include a system message followed by the conversation
    msgs = [SystemMessage(content=SYSTEM_PROMPT)] + state.get("messages", [])
    response = llm_with_tools.invoke(msgs)
    return {
        "messages": [response],
        "llm_calls": state.get("llm_calls", 0) + 1,
    }

# Tool node: execute tool calls that the LLM requested
def tool_node(state: MessagesState):
    """Perform any tool calls found on the last LLM message and return ToolMessage objects."""
    result = []
    messages = state.get("messages", [])
    if not messages:
        return {"messages": []}

    last = messages[-1]
    # In LangChain message objects, tool calls live in `.tool_calls` on the message
    tool_calls = getattr(last, "tool_calls", []) or []
    for tool_call in tool_calls:
        # tool_call expected shape: {"name": ..., "args": ...}
        tool_name = tool_call.get("name")
        args = tool_call.get("args", {})
        if tool_name not in tools_by_name:
            obs = f"Tool {tool_name} not found"
        else:
            obs = tools_by_name[tool_name].invoke(args)
        # Create a ToolMessage with the observation
        tm = ToolMessage(content=obs, tool_call_id=tool_call.get("id"))
        result.append(tm)
    return {"messages": result}

# --- Step 5: Edge decision function: continue if model made a tool call ---
def should_continue(state: MessagesState) -> Literal["tool_node", END]:
    messages = state.get("messages", [])
    if not messages:
        return END
    last = messages[-1]
    if getattr(last, "tool_calls", None):
        return "tool_node"
    return END


다음 셀에서는 helper 함수를 실행하여 추적 출력을 파일에 인쇄합니다.

In [27]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ToolMessage
def output_trace(element:str, trace, node=True):
    global trace_handle
    if trace_enabled:
        print(datetime.datetime.now(),file=trace_handle)
        print(("Node: " if node else "Edge: ")+ element, file=trace_handle)
        if element == "ask_model_to_reason (entry)":
            for single_trace in trace:
                print(single_trace, file=trace_handle)
        else:
            print(trace, file=trace_handle)
        print('----', file=trace_handle)
        
def consolidate_tool_messages(message):
    tool_messages=[]
    for msg in message:
        if isinstance(msg, ToolMessage):
            tool_messages.append(msg)
    return tool_messages

## 태스크 6.3: 에이전트 그래프 구축

이 태스크에서는 외부 도구와 상호 작용할 수 있는 대화형 AI 시스템용 에이전트 그래프를 생성하게 됩니다. 에이전트 그래프는 도구와의 대화 및 상호 작용 흐름이 정의되는 상태 머신입니다.

다음 셀에서는 입력에 따라 상태를 업데이트하는 관련 함수가 있는 노드를 정의합니다. 그래프가 한 노드에서 다음 노드로 전환되는 가장자리를 사용하여 노드를 연결합니다. 조건부 에지를 통합하여 특정 조건에 따라 그래프를 다른 노드로 라우팅합니다. 마지막으로 에이전트 그래프를 컴파일하여 정의된 대로 전환 및 상태 업데이트를 처리하여 실행을 준비합니다.

In [28]:
# --- Step 6: Build and compile the StateGraph ---
agent_builder = StateGraph(MessagesState)
agent_builder.add_node("llm_node", llm_node)
agent_builder.add_node("tool_node", tool_node)

# Connect start to llm_node
agent_builder.add_edge(START, "llm_node")
# Conditional: if llm created tool calls -> tool_node else END
agent_builder.add_conditional_edges("llm_node", should_continue, ["tool_node", END])
# After executing tools, go back to llm
agent_builder.add_edge("tool_node", "llm_node")

# Compile the agent
agent = agent_builder.compile()


In [30]:
user_messages = [HumanMessage(content="What is 5*7?"), HumanMessage(content="P001")]
initial_state = {"messages": user_messages, "llm_calls": 0}

print("Invoking agent with example messages...\n")
out = agent.invoke(initial_state)
# The agent returns a dict-like state; print messages
messages_out = out.get("messages", [])
for m in messages_out:
    try:
        m.pretty_print()
    except Exception:
        print(repr(m))

print("\nDone.")


Invoking agent with example messages...

================================ Human Message =================================

What is 5*7?
================================ Human Message =================================

P001
================================== Ai Message ==================================

[{'type': 'text', 'text': "<thinking>The User has asked for a simple multiplication calculation. The 'calculator' tool can be used to safely evaluate the expression.</thinking>\n"}, {'type': 'tool_use', 'name': 'calculator', 'input': {'expression': '5*7'}, 'id': 'tooluse_eCrh6WQdSxOQxmqGlD0Cyg'}]
Tool Calls:
  calculator (tooluse_eCrh6WQdSxOQxmqGlD0Cyg)
 Call ID: tooluse_eCrh6WQdSxOQxmqGlD0Cyg
  Args:
    expression: 5*7
================================= Tool Message =================================

35
================================== Ai Message ==================================

The result of the multiplication 5 * 7 is 35.

Done.


다음으로 컴파일된 그래프를 시각화합니다. 점선으로 표시된 것처럼 에이전트 노드에서 조건부로 전환되는 것을 관찰합니다.

In [ ]:
from IPython.display import Image, display

try:
    display(Image(react_agent.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

다음 셀에서는 helper 함수를 실행하여 그래프 출력을 인쇄합니다.

In [ ]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

다음으로 이전 노트북에서 생성한 sales.csv 파일에서 에이전트에게 제품 가격에 대해 물어보고 싶은 질문을 하나 이상 추가합니다.

In [ ]:
#list of questions
questions=[]
questions.append("How much will it cost to buy 3 units of P002 and 5 units of P003?")
#questions.append("How many units of P010 can I buy with $200?")
#questions.append("Can I buy three units of P003 with $200? If not, how much more should I spend to get three units?")
#questions.append("Prices have gone up by 8%. How many units of P003 could I have purchased before the price increase with $140? How many can I buy after the price increase? Fractional units are not pssoible.")

추론에 포함된 단계를 이해하려면 추적을 활성화합니다. 그러나, 위 목록에서 **한 개의 질문을 제외한 모든 질문을 주석 처리**하여 추적 출력을 관리하기 쉽게 유지합니다. 또는 추적을 비활성화하고 모든 질문을 실행할 수도 있습니다.

In [ ]:
trace_enabled=True

if trace_enabled:
    file_name="trace_"+str(datetime.datetime.now())+".txt"
    trace_handle=open(file_name, 'w')

다음 셀에서 위 목록의 질문으로 에이전트를 호출합니다.

In [ ]:
system_message="Answer the following questions as best you can. Do not make up an answer. Think step by step. Do not perform intermediate math calculations on your own. Use the calculator tool provided for math calculations."

for q in questions:
    inputs = {"messages": [("system",system_message), ("user", q)]}
    config={"recursion_limit": 15}
    print_stream(react_agent.stream(inputs, config, stream_mode="values"))
    print("\n"+"================================ Answer complete ================================="+"\n")

if trace_enabled:
    trace_handle.close()